In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [2]:
def make_soup(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    return soup

In [37]:
# Function to get gameIds for a particular year/week
results = {}
def get_gameId(year,week):
    # Make a soup object for the appropriate page
    url = "http://www.espn.com/nfl/schedule/_/week/{0}/year/{1}/seasontype/2".format(week,year)
    soup = make_soup(url)
    sched_page = soup.find("section",{"id":"main-container"})
    
    # Make a list for gameIds
    gameids = []
    for link in sched_page.find_all("a"):
        if "gameId" in link.get('href') and "watchespn"  not in link.get('href'):
            # Extract last bit of url listed
            s = "gameId="
            this_game = link.get('href').split(s)[1]
            gameids.append(this_game)
            # And add text displayed to a dictionary
#            results[this_game] = link.contents[0]
    
    hometeams = []
    awayteams = []
    teamnames = sched_page.find_all("a", {"class":"team-name"})
    for i, link in enumerate(teamnames):
        if "team/_/name" in link.get('href'):
            abbr = link.get('href').split("name")[1].split("/")[1]
            if i%2:
                hometeams.append(abbr)
            else:
                awayteams.append(abbr)
            
    
#    return gameids, results
    return (gameids, hometeams, awayteams)

In [38]:
gameids = []
gameresults = {}
gameyear = {}
gameweek = {}
for year in range(2018,2019):
    for week in range(1,2):
        print("Looking up gameIds for {0} week {1}".format(year,week))
        ids, home, away = get_gameId(year,week)
#        ids, results = get_gameId(year,week)
        gameids.append(ids)
        
        # Add entry in dictionaries for each gameId
        for i in ids:
            gameyear[i] = year
            gameweek[i] = week
#            gameresults[i] = results[i]
        
        # Sleep so we don't get blocked
        time.sleep(0.5)
        
# Now flatten gameids, which is a list of weekly lists
ids = [i for sublist in gameids for i in sublist]
print(gameresults)

Looking up gameIds for 2018 week 1
{}


In [42]:
# Make dataframe for game-specific information using our dictionaries for year and week
data = [ {'gameId':i,
          'season':gameyear[i],
          'week':gameweek[i]}
#          'result':gameresults[i]}
        for i in ids]
gamedata_df = pd.DataFrame(data)
# Set the gameId as the unique identifier for each row
gamedata_df.set_index('gameId', inplace=True)
gamedata_df['home'] = home
gamedata_df['away'] = away
gamedata_df.head(10)

,season,week,home,away
gameId,,,,
401030710,2018,1,phi,atl
401030718,2018,1,cle,pit
401030717,2018,1,ind,cin
401030716,2018,1,mia,ten
401030715,2018,1,min,sf
401030714,2018,1,ne,hou
401030713,2018,1,no,tb
401030712,2018,1,nyg,jax
401030711,2018,1,bal,buf
